In [513]:
import pandas as pd
import numpy as np
import plotly.express as px
import re
from siuba.dply.forcats import fct_lump, fct_reorder

%matplotlib inline

## Importing Data

In [457]:
brewing_materials = pd.read_csv('https://raw.githubusercontent.com/rfordatascience/tidytuesday/master/data/2020/2020-03-31/brewing_materials.csv')
beer_taxed = pd.read_csv('https://raw.githubusercontent.com/rfordatascience/tidytuesday/master/data/2020/2020-03-31/beer_taxed.csv')
brewer_size = pd.read_csv('https://raw.githubusercontent.com/rfordatascience/tidytuesday/master/data/2020/2020-03-31/brewer_size.csv')
beer_states = pd.read_csv('https://raw.githubusercontent.com/rfordatascience/tidytuesday/master/data/2020/2020-03-31/beer_states.csv')

## Exploring Data

The `brewing_materials` dataset looks like it shows annual and monthly quantites (lbs) of different brewing materials used to produce beer.

In [155]:
brewing_materials.head()

,data_type,material_type,year,month,type,month_current,month_prior_year,ytd_current,ytd_prior_year
0,Pounds of Materials Used,Grain Products,2008,1,Malt and malt products,374165152,365300134,374165152.0,365300134.0
1,Pounds of Materials Used,Grain Products,2008,1,Corn and corn products,57563519,41647092,57563519.0,41647092.0
2,Pounds of Materials Used,Grain Products,2008,1,Rice and rice products,72402143,81050102,72402143.0,81050102.0
3,Pounds of Materials Used,Grain Products,2008,1,Barley and barley products,3800844,2362162,3800844.0,2362162.0
4,Pounds of Materials Used,Grain Products,2008,1,Wheat and wheat products,1177186,1195381,1177186.0,1195381.0


The `beer_taxed` dataset contains data on the barrels of beer produced by year and by type of production.

In [156]:
beer_taxed.head()

,data_type,tax_status,year,month,type,month_current,month_prior_year,ytd_current,ytd_prior_year,tax_rate
0,Barrels Produced,Totals,2008,1,Production,16211480,15880125,16211480.0,15880125.0,$7/$18 per barrel
1,Barrels Produced,Taxable,2008,1,In bottles and cans,13222104,12824278,13222104.0,12824278.0,$7/$18 per barrel
2,Barrels Produced,Taxable,2008,1,In barrels and kegs,1371239,1357372,1371239.0,1357372.0,$7/$18 per barrel
3,Barrels Produced,Taxable,2008,1,"Tax Determined, Premises Use",7426,8419,7426.0,8419.0,$7/$18 per barrel
4,Barrels Produced,Sub Total Taxable,2008,1,Sub Total Taxable,14600769,14190069,14600769.0,14190069.0,$7/$18 per barrel


The `brewer_size` dataset shows the annual number of brewers by range of barrels produced. 

In [157]:
brewer_size.head()

,year,brewer_size,n_of_brewers,total_barrels,taxable_removals,total_shipped
0,2009,"6,000,001 Barrels and Over",18,1.712329e+08,1.596440e+08,3639970.0
1,2009,"1,000,001 to 6,000,000 Barrels",4,9.970404e+06,9.592723e+06,14548.0
2,2009,"500,001 to 1,000,000 Barrels",7,4.831386e+06,4.535659e+06,21563.0
3,2009,"100,001 to 500,000 Barrels",27,5.422156e+06,4.469289e+06,128000.0
4,2009,"60,001 to 100,000 Barrels",19,1.501977e+06,1.224618e+06,95732.0


Finally, `beer_states` seems to contain data on the annual amount of beer consumed (barrels) per state per production type. This could be represented nicely on a map.

In [158]:
beer_states.head()

,state,year,barrels,type
0,AK,2008,2067.69,On Premises
1,AK,2009,2263.65,On Premises
2,AK,2010,1929.15,On Premises
3,AK,2011,2251.02,On Premises
4,AK,2012,2312.43,On Premises


### What ingredients are used in US beer production?

Creating new date column for month and year values to make working with data over time easier:

In [159]:
brewing_materials["date"] = pd.to_datetime(brewing_materials["month"].astype(str) + "-" + brewing_materials["year"].astype(str))

Plotting the data for most recent month by type:

In [328]:
brewing_materials_sample = brewing_materials[brewing_materials['date'] == brewing_materials['date'].max()]
fig = px.bar(
    brewing_materials_sample,
    x='month_current',
    y='type',
    orientation='h'
)

fig.show()

Based on the above, it looks like "Total Used" is a sum of "Total grain products" and "Total non-grain products" which are sums of the individual materials by type. Removing totals and colour coding the bar chart by material type, we can see the comparison of individual materials used and their respective types.

In [329]:
brewing_materials_sample = brewing_materials_sample[brewing_materials_sample['material_type'].str.contains('Total') == False]
fig = px.bar(
    brewing_materials_sample,
    x='month_current',
    y='type',
    orientation='h',
    color='material_type'
)

fig.update_layout(yaxis={'categoryorder': 'total ascending'})

fig.show()

From the above, it is clear that malt and malt products are by far the most used material (at least in the sample month). Let's take a look at plotting these values over time.

In [330]:
bm_over_time = brewing_materials[brewing_materials['material_type'].str.contains('Total') == False]
fig = px.bar(
    bm_over_time,
    x='date',
    y='month_current',
    color='type'
)

fig.show()

Based on the above graph, it looks like there is something wrong with the data from 2016 onwards, so it might be better to filter out those values and only work with complete data. At this point, we can also sort the values within each date by total sum of each type. That way, the colors in each bar will be sorted by the overall value of each type.

We can also update the axes and legend titles to more suitable values.

In [331]:
bm_over_time = bm_over_time[bm_over_time['year'] < 2016]
bm_over_time = bm_over_time.sort_values(by='type', key=lambda x: bm_over_time['month_current'].groupby(bm_over_time['type']).transform('sum'), ascending=False)
fig = px.bar(
    bm_over_time,
    x='date',
    y='month_current',
    color='type',
)

fig.update_layout(
    legend_traceorder="reversed", 
    xaxis_title='Time',
    yaxis_title='Pounds used in beer production',
    legend_title='Material'
)

fig.show()

We can also create an alternative plot showing just the total values for the grain and non-grain categories:

In [332]:
bm_totals = brewing_materials[
    (brewing_materials['material_type'].str.contains('Total')) &
    (brewing_materials['material_type'].str.contains('Total Used') == False)
]

bm_totals = bm_totals[bm_totals['year'] < 2016]

fig = px.bar(
    bm_totals,
    x='date',
    y='month_current',
    color='type'
)

fig.update_layout(
    legend_traceorder="reversed", 
    xaxis_title='Time',
    yaxis_title='Pounds used in beer production',
    legend_title='Material'
)

fig.show()

### Brewer Size Distribution

First, we can take a look at the total barrels produced by brewer size each year.

In [613]:
bs = brewer_size[
    (brewer_size['brewer_size'].str.contains('Total') == False) &
    (np.isnan(brewer_size['total_barrels']) == False)
].copy()


def parse_first_num(text):
    words = text.split(' ')
    nums = []
    for word in words: 
        num = re.findall(r'[0-9]+', word)
        if len(num):
            nums.append(num)
            
    if len(nums):
        return "".join(nums[0])
    else:
        return "0"

bs_sort = bs['brewer_size'].apply(lambda x: int(parse_first_num(x)))
bs['brewer_size'] = fct_lump(bs['brewer_size'], 5, w=bs_sort)
fct_reorder(bs['brewer_size'], bs_sort)

fig = px.bar(
    bs,
    x='year',
    y='total_barrels',
    color='brewer_size'
)

fig.show()

We can reduce the number of brewer size options by lumping together some of the values to form a larger range:

In [537]:

fig = px.bar(
    bs,
    x='year',
    y='total_barrels',
    color='brewer_size'
)

fig.show()